# Dimensionality reduction

> This notebook gets the embeddings (or latent space) from a multivariate time series 
given by a encoder (e.g., autoencoder) and uses them as input for a 
dimensionality reduction algorithm, to generate projectsion of the embeddings

In [1]:
from tsai.imports import *
from tsai.data.preparation import SlidingWindow
from fastcore.all import *
from dvats.all import *
import wandb
api = wandb.Api()
from yaml import load, FullLoader
import hdbscan
import shap

## Config parameters

Put here everything that could be needed if this notebook

In [2]:
import nbs_pipeline.utils.config as cfg
config, job_type = cfg.get_artifact_config_dimensionality_reduction(True)

Current: /home/macu/work/nbs_pipeline
yml: ./config/04-dimensionality_reduction.yaml
Getting content./config/04-dimensionality_reduction.yaml
... About to replace includes with content
Load content./config/04-dimensionality_reduction.yaml
enc_artifact: mi-santamaria/deepvats/mvp-SWV:latest


In [4]:
for key, value in config.items():
    print(f"{key}: {value}")

use_wandb: True
wandb_group: None
wandb_entity: mi-santamaria
wandb_project: deepvats
valid_artifact: None
train_artifact: mi-santamaria/deepvats/mvp-SWV:latest
n_neighbors: 15
min_dist: 0.1
random_state: 1234


This model needs to restore the encoder model fitted in the notebook `02x`, as well as the data and configuration

## Run

In [5]:
import os
path = os.path.expanduser("~/work/nbs_pipeline/")
name="04_dimensionality_reduction"
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"

In [6]:
if config.valid_artifact is not None:
    runname="04_dimensionality_reduction | train: "+ config.train_artifact + " | valid: "+config.valid_artifact
else:
    runname="04_dimensionality_reduction | train: "+ config.train_artifact + " | valid: None"
    
    
print("Runname: "+ runname)
run_dr = wandb.init(
    entity              = config.wandb_entity,
    project             = config.wandb_project if config.use_wandb else 'work-nbs', 
    group               = config.wandb_group,
    allow_val_change    = True, 
    job_type            = job_type, 
    mode                = 'online' if config.use_wandb else 'disabled',
    anonymous           = 'never' if config.use_wandb else 'must',
    config              = config,
    resume              = False,
    name                = runname
)
config_dr = wandb.config # Object for storing hyperparameters

Runname: 04_dimensionality_reduction | train: mi-santamaria/deepvats/mvp-SWV:latest | valid: None


wandb: Currently logged in as: mi-santamaria. Use `wandb login --relogin` to force relogin


'stream.Stream' object attribute 'write' is read-only


In [7]:
# Botch to use artifacts offline
artifacts_gettr = run_dr.use_artifact if config_dr.use_wandb else api.artifact

Restore the encoder model and its associated configuration

In [8]:
enc_artifact = artifacts_gettr(
    'mi-santamaria/deepvats/embeddings-SWV:latest', 
    type='object')
enc_artifact.metadata

{'cpu': False,
 'ref': {'hash': '-4811609136384432385', 'type': "<class 'numpy.ndarray'>"},
 'input_ar': 'mi-santamaria/deepvats/solar_4_seconds:v0',
 'use_wandb': True,
 'wandb_group': 'embeddings',
 'enc_artifact': 'mi-santamaria/deepvats/mvp-SWV:latest',
 'wandb_entity': 'mi-santamaria',
 'wandb_project': 'deepvats'}

In [9]:
# TODO: This only works when you run it two timeS! WTF?
try:
    embs = enc_artifact.to_obj()
except:
    embs = enc_artifact.to_obj()
embs

wandb: Downloading large artifact embeddings-SWV:latest, 240.79MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.7


array([[8.9753598e-02, 7.8738242e-01, 8.7640753e+00, ..., 2.9195672e-01,
        0.0000000e+00, 4.2660935e-03],
       [8.9753598e-02, 7.8738242e-01, 8.7640753e+00, ..., 2.9195672e-01,
        0.0000000e+00, 4.2660935e-03],
       [8.9753598e-02, 7.8738242e-01, 8.7640753e+00, ..., 2.9195672e-01,
        0.0000000e+00, 4.2660935e-03],
       ...,
       [8.9753598e-02, 7.8738242e-01, 8.7640753e+00, ..., 2.9195672e-01,
        0.0000000e+00, 4.2660935e-03],
       [8.9753598e-02, 7.8738242e-01, 8.7640753e+00, ..., 2.9195672e-01,
        0.0000000e+00, 4.2660935e-03],
       [8.9753598e-02, 7.8738242e-01, 8.7640753e+00, ..., 2.9195672e-01,
        0.0000000e+00, 4.2660935e-03]], dtype=float32)

Restore the dataset artifact used for training the encoder. Even if we do not compute the dimensionality reduction over this dataset, we need to know the metadata of the encoder training set, to check that 
it matches with the dataset that we want to reduce.

In [ ]:
dr_methods = ['UMAP', 'TSNE', 'PCA']
dr_method = dr_methods[0]
print(dr_method)
match dr_method:
    case 'UMAP':
        print("You selected UMAP")
        res=get_UMAP_prjs(input_data = embs, cpu=F, random_state=1234),
    case 'TSNE':
        print("You selected TSNE")
        res=get_TSNE_prjs(X = embs, cpu=F, random_state=1234)
    case 'PCA':
        print("You selected PCA")
        res=get_PCA_prjs(X = embs, cpu=F, random_state=1234)
    case _:
        print("No way... Say UMAP")
        res=get_UMAP_prjs(input_data = embs, cpu=F, random_state=1234),
    
res = res.to_df() # TODO: This should be a matrix for improved efficiency
#colnames(res) = c("xcoord", "ycoord")
#on.exit(print(" prj_object -->"))
res

UMAP
You selected UMAP


Now we need to check whether the artifact that is going to be used fort the dimensionality reduction matches the artifact used to train the encoder. Matching means having the same number of variables, the same window size and stride, and the same frequency.

In [ ]:
df = dr_artifact.to_df()
df

In [ ]:
df.shape

In [ ]:
enc_input, _ = SlidingWindow(window_len=enc_logger.config['w'], 
                             stride=enc_logger.config['stride'], 
                             get_y=[])(df)
enc_input.shape

### Getting the embeddings (activations) from the encoder

In [ ]:
embs = get_enc_embs(enc_input, enc_learner, cpu=False)
test_eq(embs.shape[0], enc_input.shape[0])
embs.shape

Average embeddings in the time dimension, if needed

## Dimensionality reduction using UMAP

We use DR techniques to provide an alternative view for users to visually analyze and explore the time-series data. The algorithm UMAP shows its high competitiveness compared to t-SNE. t-SNE suffers from some limitations such as loss of large-scale information (the inter-cluster relationships). UMAP has a faster runtime and provides better scaling which helps to gain a meaningful organization of clusters, outliers and the preservation of continuums compared to t-SNE

For this part of the implementation, the package [umap-learn](https://github.com/lmcinnes/umap) is used. The input of the algoritm is the $n \times \delta$ that contains, for each slice of the time series, the corresponding $\delta$ latent embeddings given by the encoder.

The hyperparameters of UMAP are given values by default here. If the value has been already set previously, that means this notebook is being called from a wandb sweep, and we must use the value that the sweep is bringing.

In [ ]:
umap_params = {
    'n_neighbors' : config_dr.n_neighbors,
    'min_dist' : config_dr.min_dist,
    'random_state': config_dr.random_state
}

In [ ]:
prjs = get_UMAP_prjs(embs, cpu=False, **umap_params)
prjs.shape

Save the projections as an artifact

In [ ]:
if config.use_wandb: 
    run.log_artifact(ReferenceArtifact(prjs, 'projections', type='projections', 
metadata=dict(run_dr.config)), aliases=f'run-{run.project}-{run.id}')

## Create Precomputed Clusters

In order to integrate precomputed clusters into the embedding space, it's necessary to log artifacts that include the labels of the newly created clusters. 

The cluster creation process is presented below. This creation procedure can be modified according to specific needs. However, the structure of the new artifact must be preserved (it must be a numpy.ndarray and the number of elements must be equal to the number of points in the embedding space).

In [ ]:
print(f'HDBSCAN supported metrics: {list(hdbscan.dist_metrics.METRIC_MAPPING.keys())}')

In [ ]:
# Define HDBSCAN parameters
hdbscan_kwargs = {
    'min_cluster_size' : 100,
    'min_samples' : 15,
    'cluster_selection_epsilon' : 0.08,
}
metric_kwargs = {
    'metric' : 'jaccard'
}

In [ ]:
# Create clusters using HDBSCAN
clusters = hdbscan.HDBSCAN(**hdbscan_kwargs, **metric_kwargs).fit(prjs)
clusters_labels = clusters.labels_
clusters_labels

In [ ]:
# Testing artifact structure 
test_eq_type(type(clusters_labels), np.ndarray)
test_eq(clusters_labels.size, prjs.shape[0])

In [ ]:
# Create and log 'clusters_labels' artifact
clusters_ar = ReferenceArtifact(obj=clusters_labels, name='clusters_labels')
clusters_ar.metadata, clusters_ar.manifest.entries.values()

In [ ]:
run_dr.log_artifact(clusters_ar, aliases=['hdbscan_jaccard'])

## 2D Visualization 

While the connected scatter plot is a simple visualization technique, it has very specific functions in our approach. Every sliding window is represented as a dot in the plot after the projection process (Fig. 4C, D of the paper). Before labeling, all points have the same color and transparency, and when they are concentrated in one area, the densities are accumulated. Lines are used to connect consecutive points preserving the temporal ordering of the data and allowing the user to see temporal connections (Fig. 4B of the paper). Thus, the point is linked to the previous point (inner) and to the posterior point (outer) as an indication of the flow of time.

In [ ]:
#export
def plot_projections(prjs, umap_params, fig_size = (25,25)):
    "Plot 2D projections thorugh a connected scatter plot"
    df_prjs = pd.DataFrame(prjs, columns = ['x1', 'x2'])
    fig = plt.figure(figsize=(fig_size[0],fig_size[1]))
    ax = fig.add_subplot(111)
    ax.scatter(df_prjs['x1'], df_prjs['x2'], marker='o', facecolors='none', edgecolors='b', alpha=0.1)
    ax.plot(df_prjs['x1'], df_prjs['x2'], alpha=0.5, picker=1)
    plt.title('DR params -  n_neighbors:{:d} min_dist:{:f}'.format(
        umap_params['n_neighbors'],umap_params['min_dist']))
    return ax

In [ ]:
prjs_plt = plot_projections(prjs, umap_params)

In [ ]:
prjs_plt = plot_projections(prjs, umap_params)

Log this plot as part of the current wandb run

In [ ]:
#hide

# Get the figure of the embedding plot, and save it on thea wandb run.
run_dr.log({"img": [wandb.Image(prjs_plt.get_figure(), caption="dr_projections_plot")]})

#run_dr.log({'embeddings_plot': embeddings_plot})

In [ ]:
#hide
run_dr.finish()

# Explainability with SHAP (future work)

In [ ]:
# %matplotlib inline

# fig = plt.figure(figsize=(10,10))
# ax = fig.add_subplot(111)

# ax.scatter(df_embeddings['x1'], df_embeddings['x2'], marker='o', facecolors='none', edgecolors='b', alpha=0.1)
# ax.plot(df_embeddings['x1'], df_embeddings['x2'], alpha=0.5, picker=1)
# ax.set_title('Select the point you want to visualize as a time window in the original space')

Make the plot interactive to allow selection of subsets of the plot

In [ ]:
# selected_indices = None
# selected_points = None

# def onpick(event):
#     global selected_points
#     thisline = event.artist
#     xdata = thisline.get_xdata()
#     ydata = thisline.get_ydata()
#     global selected_indices
#     selected_indices = event.ind
#     selected_points = tuple(zip(xdata[selected_indices], ydata[selected_indices]))
#     print('onpick points (first):', selected_points[0])

# fig.canvas.mpl_connect('pick_event', onpick)

# plt.show()
# fig.tight_layout()
# fig.savefig(f'../img/w={w}.png')

## Hyperparameter tuning for the dimensionality reduction

There are a number of parameters that can be set for the UMAP algorithm. The major 
ones are `n_neighbors` and `min_dist`. Thus, we will carry out a hyperparameter 
sweep in Weights and Biases for these two parameters. Note that there is no objective
way of deciding that some embeddings are better than others. Thus, we must rely on our
intuition by visualizing the 2D plots of each of the runs in the sweep.

The first thing we need is gather all the pipeline of the previous section into a function


## Linking back points of the 2D projection to the original time series

The variable `selected_points` and `ind` contain an array of the points and indices selected in the previous 2D projection. We will take the first of them (there can be many selected points with just one click), and use its index to get the corresponding time window of the original space.

In [ ]:
# selected_window = input_data[df_embeddings.sample(n=1).index][0] if selected_indices is None else input_data[selected_indices[0]]
# selected_window

### Visualizing all the variables in the time window (default)

In [ ]:
# %matplotlib inline
# g = sns.FacetGrid(df_output_tidy, col="variable", col_wrap=3, aspect=2)
# g = g.map(plt.plot, "timestamp", "value")

### Contribution: Visualize only the most relevant variables

In high dimensional time series, not only is interesting to see the window associated to a point in the 2D space, but also it is extremely important to spot which variables are mainly causing that the window is positioned in that point of the 2D space.

Since UMAP does not provide capabilities to understand feature importance, there are [different ways](https://stats.stackexchange.com/questions/438025/understand-important-features-in-umap) to tackle this problem:

1. Use another dimensionality reduction technique that provides importance, such as [sparse PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.SparsePCA.html)

2. Create a surrogate model on top of the inputs/output of UMAP and explain it using XAI techniques. We will try here this option.

The idea is to have a surrogate model that takes the multivariate time series as input and produces the associated points in the 2D space as ouput. Since we already have a Deep Convolutional Autoencoder (DCAE) that takes a multivariate time series as input, and it contains the latent features that represent that input, we can use it for the surrogate. We will use the intermediate model that goes from the input to the layer containing the latent space, and then add a `Dense` layer with 2 units and linear activation.

In [ ]:
#export
# def train_surrogate_model(dcae, embeddings, lat_ln='latent_features'):
#     "Train a surrogate model that learns the `embeddings` from the latent features contained in the layer \
#     `lat_ln` of a previously trained Deep Convolutional AutoEncoder `dcae`"
#     x = dcae.get_layer(lat_ln).output
#     x = Dense(units=embeddings.shape[1], activation='linear')(x)
#     surrogate_model = Model(dcae.input, x)
#     l_nms = [layer.name for layer in surrogate_model.layers]
#     layer_idx = l_nms.index(lat_ln)
#     # The layers that are already trained from the autoencoder must be `frozen`
#     for layer in surrogate_model.layers[:layer_idx]:
#         layer.trainable = False
#     return surrogate_model

In [ ]:
# sm = train_surrogate_model(m, embeddings, lat_ln='latent_features')

In [ ]:
# test.equals(sm.input.shape, m.input.shape)
# test.equals(sm.output.shape[1], embeddings.shape[1])
# l_nms = [layer.name for layer in sm.layers]
# layer_idx = l_nms.index('latent_features')
# test.all_equal([layer.trainable for layer in sm.layers], \
#                np.repeat([False, True], [layer_idx + 1, len(sm.layers) -1 -layer_idx]))

Configure the training hyperparameters

In [ ]:
# loss = 'mean_squared_error'
# opt = 'adam'
# bs = 100
# epochs = 10
# val = .2

In [ ]:
# sm.output_shape

In [ ]:
# input_data.shape

In [ ]:
# embeddings.shape

In [ ]:
# sm.fit(x=input_data, y=embeddings, batch_size=bs, validation_split=val, epochs=epochs, callbacks=[WandbCallback()])

In [ ]:
# import innvestigate

In [ ]:
# analyzer = innvestigate.create_analyzer("gradient", intermediate_model)

In [ ]:
# asd= innvestigate.create_analyzer("gradient", m)

In [ ]:
# from keras.datasets import mnist

In [ ]:
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# x_train.shape

In [ ]:
# input_data[np.random.choice(input_data.shape[0], 100, replace=False)].shape

In [ ]:
# import shap

In [ ]:
# background = input_data[np.random.choice(input_data.shape[0], 100, replace=False)]
# e = shap.DeepExplainer(intermediate_model, background)

In [ ]:
# shap_values = e.shap_values(input_data[0:4])

In [ ]:
# shap_values[0][0]